To use this script:
1. Save the requirements tab in a spreadsheet as a UTF-8 CSV (note: not a normal CSV; must specify UTF-8)
2. When calling ingestRFP, indicate start_row = the number of the row where the requirements start
3. end_row = the last row to read in
4. req_i = the index (number) of the column with the requirements
5. rat_i = the index (number) of the column with the ratings we specify
6. resp_i = the index (number) of the column with the responses we specify
7. Note that the column header gets overwritten for the requirement, rating, and response columns. These should be changed back.
8. The script adds a column called "match" which tells you how strong of a match the requirement was to a past requirement. A value of 1.00 means it was an exact match. The answers get very unreliable when the match is less than 0.9, so read those extra-carefully. 

In [35]:
# read in CSVs of requirements, ratings, and responses as a dataframe
# Cobb-specific
import numpy as np
import pandas as pd

def ingestRFP(file, start_row = 0, end_row = 999, req_i = 2, rat_i = 5, resp_i =6):
    '''
    filelist is a file path to a CSV for a proposal
    req_i is the column index for requirements
    resp_i is the column index for responses
    start_row is the row to start reading on
    end_row is the row to stop reading on
    outputs a dataframe of requirement-response pairs
    
    '''
    
    # NOTE: in Excel, save as CSV UTF-8
    rfp_df = pd.read_table(file, 
                           sep = ",", 
                           skiprows = (start_row-1), 
                           nrows = (end_row - start_row), 
                           skipinitialspace = True)

    # df.rename(columns={ df.columns[1]: "whatever" })
    
    # rename the columns to standardize
    rfp_df = rfp_df.rename(columns = {
        rfp_df.columns[req_i]: 'req',
        rfp_df.columns[rat_i]: 'rating',
        rfp_df.columns[resp_i]: 'resp',
    })

    return rfp_df

corpus = ingestRFP('C:/Users/ahicken/Documents/proposal_data/naedutf8.csv', 8, 230, 2, 6, 7)
# last row should be 230

newRFP = ingestRFP('C:/Users/ahicken/Documents/proposal_data/adha.csv', 8, 132, 2, 5, 6)
# last row should be 132

newRFP

,Item,Functional Area,req,ADHA Priority,Notes to Vendor,rating,resp
0,2,Educational Content,Ability to import and launch non-SCORM complia...,M,"This content might include, for example, recor...",NaN,NaN
1,3,Educational Content,Ability to configure one or more types of digi...,M,"This might include, for example, combining mul...",NaN,NaN
2,4,Educational Content,Ability to manage registration and access for ...,M,ADHA currently uses Zoom. It should be possibl...,NaN,NaN
3,7,Educational Content,"Ability to create/configure extended, facilita...",M,"For example, it should be possible to configur...",NaN,NaN
4,8,Educational Content,Ability to set enrollment durations for courses,M,Please specify how enrollment durations can be...,NaN,NaN
5,9,Educational Content,Ability to configure and enforce a certificati...,M,"This should include, for example, the ability ...",NaN,NaN
6,11,Educational Content,Ability to assign documents from a central doc...,M,Updating a document in the central repository ...,NaN,NaN
7,12,Educational Content,Ability to offer closed discussion forums that...,M,It should be possible to associate a discussio...,NaN,NaN
8,17,Educational Content,Ability to associate multiple assessments with...,M,This applies to both on-demand and facilitated...,NaN,NaN
9,18,Educational Content,Ability to block access to course content unti...,M,NaN,NaN,NaN


In [36]:
import difflib
# difflib.SequenceMatcher(None, 'hello world', 'hello').ratio()

def respondToReq(req, corpus):
    '''
    Given a requirement (req.req) and a dataframe 'corpus' of past responses
    Read through the corpus (corpus.req) to find the past requirement that most closely matches the requirement
    Overwrite the response (req.resp) and rating (req.rating) with that requirement's response.
    '''
    
    # req.match exists to hold the match ratio which we get from the 'difflib' library
    req.match = 0
    
    print("...processing requirement ", req.req)
    
    for index, row in corpus.iterrows():
        # this generates a number between 0 and 1 with the similarity of two strings
        try:
            ratio = difflib.SequenceMatcher(None, row.req, req.req).ratio()
        except: 
            ratio = 0
            # print("SequenceMatcher choked on row.req = ", row.req, " , req.req = ", req.req)
        
        # if this is the highest ratio we've found, update the requirement row to match the corpus row
        if ratio > req.match:
            req.resp = row.resp
            req.rating = row.rating
            req.match = ratio
    
    return req

def output_response(corpus, RFP):
    '''
    Given a 'corpus' of past answers
    output an RFP response as a dataframe
    that duplicates previous responses to all exactly matching requirements 
    match on requirement language
    '''
    
    # add a column to the response RFP to hold the match ratio
    RFP['match'] = pd.Series()
    
    # read through each line of the RFP, filling in responses to requirements
    filledRFP = RFP.apply(respondToReq, corpus = corpus, axis = "columns")
    
    return filledRFP


filledRFP = output_response(corpus, newRFP)

filledRFP.to_csv("adha_filled.csv", na_rep = "", index = False, encoding = 'utf-8')

...processing requirement  Ability to import and launch non-SCORM compliant content
...processing requirement  Ability to import and launch non-SCORM compliant content
...processing requirement  Ability to configure one or more types of digital content into a course experience
...processing requirement  Ability to manage registration and access for live Webinar
...processing requirement  Ability to create/configure extended, facilitated online courses (e.g., a course an instructor leads over a period weeks using discussion, readings, assignments, etc.)
...processing requirement  Ability to set enrollment durations for courses
...processing requirement  Ability to configure and enforce a certification or learning path involving multiple courses/activities. 
...processing requirement  Ability to assign documents from a central document repository to multiple courses
...processing requirement  Ability to offer closed discussion forums that can only be accessed based upon permission being 

...processing requirement  Ability to assemble media objects and files (e.g., PDF, PPT, and Word) into distinct Web-based courses
...processing requirement  Ability to embed media objects such as videos into Web-based courses
...processing requirement  Ability to incorporate quizzes or tests into a course, preferably pulled from a central item bank of questions that is also categorized based on topic, level of education, audience and learning objectives
...processing requirement  Ability to have a workflow in place to help facilitate course creation that steps staff through the development of a course (i.e., a course creation wizard)
...processing requirement  The ability  to allow multiple users to edit, review, and approve content
...processing requirement  Ability to configure and manage live, place-based courses in the system
...processing requirement  Real-time chat tool for communication between learners and instructors
...processing requirement  Other peer-to-peer communication 

In [5]:
# TODO
# fix ingest so it doesn't use dicts 
# add date of response
# add Health / Trade / Other